In [1]:
from alinea.adel.adel_dynamic import AdelDyn
from openalea.plantgl.all import Viewer, Scene
from openalea.mtg import MTG
import os
from alinea.adel.geometric_elements import Leaves
from alinea.adel.newmtg import internode_elements, sheath_elements, blade_elements, convert, properties_from_dict, adel_metamer
import pandas as pd

In [2]:
INPUTS_DIRPATH = 'inputs'
# Create AdelDyn object and empty mtg
adel_wheat = AdelDyn()
ADELWHEAT_INPUTS_DIRPATH = os.path.join('outputs')
g = adel_wheat.load(dir=ADELWHEAT_INPUTS_DIRPATH)

In [60]:
g.vertices(1)

[1]

In [76]:
vtxList = [k for k,v in g.property('edge_type').items() if v=='/']
names = [ g.property('label')[vid] for vid in vtxList ]
print(names)

['plant1', 'MS', 'metamer0', 'collar', 'baseElement']


In [3]:
g.display()

MTG : nb_vertices=178, nb_scales=6
/plant1			(id=1)											
^/MS			(id=2)											
^/metamer0			(id=3)											
^/collar			(id=4)											
^/baseElement			(id=5)											
^<topElement			(id=6)											
^<metamer1			(id=7)											
^/internode			(id=8)											
^/baseElement			(id=9)											
^<HiddenElement			(id=108)											
^<StemElement			(id=107)											
^<topElement			(id=10)											
	+sheath			(id=11)										
	^/baseElement			(id=12)										
	^<HiddenElement			(id=110)										
	^<StemElement			(id=109)										
	^<topElement			(id=13)										
	^<blade			(id=14)										
	^/baseElement			(id=15)										
	^<HiddenElement			(id=111)										
	^<LeafElement1			(id=157)										
	^<topElement			(id=16)										
^<metamer2			(id=17)											
^/internode			(id=18)											
^/baseElement			(id=19)											
^<HiddenElement			(id=113)											
^<StemElement			(id=112)											
^<topElement			(id=20)											
	+sheath			(id=21)									